<a href="https://colab.research.google.com/github/kushalswami20/aichatbotpyhtonqa/blob/main/Aichatbot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load the tables
questions = pd.read_csv("/content/drive/MyDrive/dataset/Questions.csv.zip", encoding="latin1")
answers = pd.read_csv("/content/drive/MyDrive/dataset/Answers.csv.zip", encoding="latin1")
tags = pd.read_csv("/content/drive/MyDrive/dataset/Tags.csv.zip", encoding="latin1")

In [ ]:
# Merge Questions and Answers
qa_pairs = pd.merge(
    questions[["Id", "Title", "Body"]],  # Select relevant columns from Questions
    answers[["ParentId", "Body"]],       # Select relevant columns from Answers
    left_on="Id",                        # Join on question ID
    right_on="ParentId",                 # Join on answer's ParentId
    suffixes=("_question", "_answer")    # Add suffixes to distinguish columns
)

# Rename columns for clarity
qa_pairs.rename(columns={"Body_question": "Question", "Body_answer": "Answer"}, inplace=True)

# Drop unnecessary columns
qa_pairs = qa_pairs[["Question", "Answer"]]

In [ ]:
from bs4 import BeautifulSoup
import re

def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove special characters and extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Apply cleaning to both columns
qa_pairs["Question"] = qa_pairs["Question"].apply(clean_text)
qa_pairs["Answer"] = qa_pairs["Answer"].apply(clean_text)

In [ ]:
# Filter high-quality questions and answers
qa_pairs = qa_pairs[(questions["Score"] > 5) & (answers["Score"] > 5)]

In [ ]:
qa_pairs.drop_duplicates(subset=["Question", "Answer"], inplace=True)

In [ ]:
qa_pairs["input_text"] = "question: " + qa_pairs["Question"]
qa_pairs["target_text"] = qa_pairs["Answer"]

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(qa_pairs, test_size=0.1, random_state=42)

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def tokenize_data(df):
    inputs = tokenizer(df["input_text"].tolist(), max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    targets = tokenizer(df["target_text"].tolist(), max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    return inputs, targets

train_inputs, train_targets = tokenize_data(train_df)
val_inputs, val_targets = tokenize_data(val_df)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_inputs,
    eval_dataset=val_inputs,
)

# Start fine-tuning
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./python_chatbot_t5")
tokenizer.save_pretrained("./python_chatbot_t5")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kushalarogyaswami_23it089 (kushalarogyaswami_23it089-delhi-technological-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./python_chatbot_t5")
tokenizer = T5Tokenizer.from_pretrained("./python_chatbot_t5")

OSError: Incorrect path_or_model_id: './python_chatbot_t5'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
import torch
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, encodings, targets):
        self.encodings = encodings
        self.targets = targets

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.targets["input_ids"][idx])
        return item

train_dataset = QADataset(train_inputs, train_targets)
val_dataset = QADataset(val_inputs, val_targets)


In [ ]:
from transformers import T5ForConditionalGeneration

# Load T5-small model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()


<ipython-input-12-f47fd5d37516>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-f47fd5d37516>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.targets["input_ids"][idx])
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.904500,2.621542
2,2.762900,2.566067
3,2.820300,2.551986


<ipython-input-12-f47fd5d37516>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-f47fd5d37516>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.targets["input_ids"][idx])
<ipython-input-12-f47fd5d37516>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-f47fd5d37516>:14: UserWarning: To copy construct from a tensor, it

TrainOutput(global_step=2943, training_loss=3.1067635357643204, metrics={'train_runtime': 1040.4582, 'train_samples_per_second': 22.623, 'train_steps_per_second': 2.829, 'total_flos': 3185675323047936.0, 'train_loss': 3.1067635357643204, 'epoch': 3.0})

In [ ]:
import os

# ✅ Define checkpoint directory
checkpoint_dir = "./checkpoints"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# ✅ Save the model and tokenizer
model.save_pretrained(checkpoint_dir)
tokenizer.save_pretrained(checkpoint_dir)

print(f"✅ Model and tokenizer saved at {checkpoint_dir}")


✅ Model and tokenizer saved at ./checkpoints


In [ ]:
def generate_answer(question):
    input_text = "question: " + question
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).input_ids
    input_ids = input_ids.to(device)

    output = model.generate(input_ids, max_length=128)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

# Test with a sample question
sample_question = "How to install pandas in Python?"
print("Generated Answer:", generate_answer(sample_question))


Generated Answer: You can install pandas in Python.


In [ ]:
pip install gradio transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 835.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gradio as gr
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# ✅ Load Trained Model & Tokenizer
checkpoint_dir = "./checkpoints"  # Adjust if needed
model = T5ForConditionalGeneration.from_pretrained(checkpoint_dir)
tokenizer = T5Tokenizer.from_pretrained(checkpoint_dir)

device = torch.device("cuda")
model.to(device)

# ✅ Define Function to Generate Answers
def generate_answer(question):
    input_text = f"question: {question}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

    output_ids = model.generate(input_ids, max_length=100)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return answer

# ✅ Create Gradio Interface
iface = gr.Interface(
    fn=generate_answer,
    inputs=gr.Textbox(label="Enter Your Question"),
    outputs=gr.Textbox(label="Generated Answer"),
    title="T5 Question Answering",
    description="Enter a question, and the T5 model will generate an answer.",
)

# ✅ Launch App in Colab
iface.launch(share=True)  # share=True generates a public URL


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d6e2607f1dbf10fd6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
print(questions.head())  # See if questions and answers look properly formatted



    Id  OwnerUserId          CreationDate  Score  \
0  469        147.0  2008-08-02T15:11:16Z     21   
1  502        147.0  2008-08-02T17:01:58Z     27   
2  535        154.0  2008-08-02T18:43:54Z     40   
3  594        116.0  2008-08-03T01:15:08Z     25   
4  683        199.0  2008-08-03T13:19:16Z     28   

                                               Title  \
0  How can I find the full path to a font from it...   
1            Get a preview JPEG of a PDF on Windows?   
2  Continuous Integration System for a Python Cod...   
3     cx_Oracle: How do I iterate over a result set?   
4  Using 'in' to match an attribute of Python obj...   

                                                Body  
0  <p>I am using the Photoshop's javascript API t...  
1  <p>I have a cross-platform (Python) applicatio...  
2  <p>I'm starting work on a hobby project with a...  
3  <p>There are several ways to iterate over a re...  
4  <p>I don't remember whether I was dreaming or ...  
